In [1]:
import Pkg; Pkg.activate(".")

  Activating new project at `~/Julia-QC/TreeStructureOpt`


In [2]:
using Revise, Graphs, Coarsening, MatrixMarket, Random, SparseArrays, GraphIO, DataStructures, Logging, LinearOrdering, Statistics, TickTock, DataFrames

In [3]:
includet("./cpu-parallel-dynamictreewidth.jl")

In [4]:
onesum = PSum(1)

PSum(1)

In [5]:
twosum = PSum(2)

PSum(2)

In [21]:
circuitfiles = [
    "./graphs/regular$(d)_32_$(p)_$(seed).mtx" for d in 3:5 for p in 2:5 for seed in 0:9
];

In [22]:
circuitadj = makeadj.(mmread.(circuitfiles));

In [32]:
circuits = SimpleGraph.(circuitadj);

In [33]:
results = []
    @showprogress 1 "Graph..." for (q, G) in enumerate(circuits)
        besttw = Inf
        bestonesumval = Inf
        @showprogress 1 "Order..." for r in 5
            @showprogress 1 "Seed..." for i in 1:20
                config = (
                    compat_sweeps = 10,
                    stride_percent = 0.5, 
                    gauss_sweeps = 10,
                    coarsening = VolumeCoarsening(0.4, 2.0, r),
                    coarsest = 10,
                    pad_percent = 0.05,
                    node_window_sweeps = 10,
                    node_window_size = 1,
                    seed = i
               )
                position_to_idx, idx_to_position = ordergraph(onesum, G; config...);
                onesumval = LinearOrdering.evalorder(onesum, adjacency_matrix(G), idx_to_position)
                tw, _ = recursive_width(adjacency_matrix(G), position_to_idx, idx_to_position)
                if tw <= besttw
                    besttw = tw
                    bestonesumval = onesumval
            end
        end
    end
end


LoadError: BoundsError: attempt to access 191-element Vector{Float64} at index [192]